In [1]:
import sys, importlib
import ncbi_crassvirales as ncbi_crassvirales
import utils as utils
from ete3 import RectFace, faces

/home/dani/miniconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?
/home/dani/miniconda3/lib/python3.9/site-packages/ete3-3.1.2-py3.7.egg/ete3/evol/parser/codemlparser.py:221: SyntaxWarning: "is" with a literal. Did you mean "=="?


In [2]:
summaries_dir = "/home/dani/encode_home/projects/Bas_phages/5_nr_screening/2_hits_summary/"
#summaries_dir = "~/projects/Bas_phages/5_nr_screening/2_hits_summary/"

#trees_dir = "/home/dani/encode_home/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/2_trees/"
#trees_dir = "/home/danielc/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/2_trees/"
#trees_dir = "/home/danielc/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/"
trees_dir = "/home/dani/encode_home/projects/Bas_phages/5_nr_screening/4_merged_ncbi_crassvirales/2_trees"


In [3]:
# get interesting CLs to look at
#importlib.reload(ncbi_crassvirales);
#ncbi_crassvirales.get_interesting_cls(trees_dir, summaries_dir)

In [45]:
#cl_id = "cl_0914"
cl_id = "cl_2097"
cl_id = "cl_s_208"
cl_id = "cl_5520"
cl_id = "cl_0958"
cl_id = "cl_6585"
cl_id = "cl_s_336"
cl_id = "cl_1285"
cl_id = "cl_s_202"
cl_id = "cl_4908"
cl_id = "cl_1635" #
cl_id = "cl_s_464"
cl_id = "cl_0163"
cl_id = "cl_0914"
cl_id = "cl_s_084"
cl_id = "cl_s_752"
cl_id = "cl_2660"
cl_id = "cl_0227"


In [46]:
# parse ncbi summary table
importlib.reload(ncbi_crassvirales);
ncbi_summary_df = ncbi_crassvirales.parse_ncbi_summary_table(cl_id, summaries_dir)
#ncbi_summary_df.loc["HDS29336.1",]

In [47]:
# parse crassvirales taxonomy 
importlib.reload(utils);

crassvirales_taxonomy_file = "crassphages_taxonomy_terL_and_new.txt"
crassvirales_taxonomy_df = utils.get_taxonomy_df(crassvirales_taxonomy_file)

In [48]:
# parse and annotate the ncbi & Crassvirales tree
importlib.reload(ncbi_crassvirales);

t = ncbi_crassvirales.read_and_annotate_tree(cl_id, trees_dir, ncbi_summary_df, crassvirales_taxonomy_df)

t = ncbi_crassvirales.add_phylum_percent_inner_nodes(t)
#t = ncbi_crassvirales.collapse_nodes_phylum_perc(t, 0.9)
t = ncbi_crassvirales.color_format_tree(t)
t.show()

In [30]:
def resolve_crass_bacteria_polytomies(t):
    '''
    Looks for clades with Crassvirales and Bacteria, all of them with 0 dist
    '''
    checked_leaves = list()
    crass_polytom_leaves = t.search_nodes(dist=0.0, superkingdom="crassvirales")
    for leaf in crass_polytom_leaves:
        if leaf.name not in checked_leaves:
            checked_leaves.append(leaf.name)
            # go one node up, check superkingdom of descendants
            node_up = leaf.up
            for node in node_up.iter_descendants():
                if node.superkingdom == "Bacteria":
                    node.superkingdom = "bact_crass_polytomy"
                    print(f"polytomy bacteria-crass, {node.name}")

    return t
    
t = resolve_crass_bacteria_polytomies(t)

In [31]:
taxa_groups = [["crassvirales","NA", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA", "Viruses"],
               ["crassvirales", "new", "NA", "bact_crass_polytomy"],
               ["crassvirales", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA"],
               ["crassvirales", "new", "Viruses"],
               ["crassvirales", "new", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses"],
               ["crassvirales", "NA", "bact_crass_polytomy"],
               ["crassvirales", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new"],
               ["crassvirales", "NA"],
               ["crassvirales", "Viruses"],
               ["crassvirales", "bact_crass_polytomy"],
               ["crassvirales"]]
               
monophyletic_clades_leaves = dict()
monophyletic_clades = dict()
mrcas_ids_to_clean = list()
cont = 0
for taxa_group in reversed(taxa_groups):
    mrcas = t.get_monophyletic(target_attr="superkingdom", values=taxa_group)
    if mrcas:
        for mrca in mrcas:
            cont += 1
            mrca_id = f"mrca_{cont}"
            # get names of the leaves
            mrca_leaves_names = {leaf.name for leaf in mrca.iter_leaves()}
            if monophyletic_clades_leaves:
                for target_mrca, target_leaves_names in monophyletic_clades_leaves.items():
                    tmp = target_leaves_names.union(mrca_leaves_names)
                    if len(tmp) == len(mrca_leaves_names):
                        mrcas_ids_to_clean.append(target_mrca)
                    
            monophyletic_clades_leaves[mrca_id] = mrca_leaves_names
            monophyletic_clades[mrca_id] = mrca
                        
mrcas_ids_to_clean   = list(set(mrcas_ids_to_clean))
for mrca_id in mrcas_ids_to_clean:
    del monophyletic_clades[mrca_id]
    del monophyletic_clades_leaves[mrca_id]


for mrca_id, mrca in monophyletic_clades.items():
    face = RectFace(5,5, "red","red", label=mrca_id)
    mrca.add_face(face, 0, "float")
print(monophyletic_clades)

{'mrca_78': Tree node '' (0x7f89ac34097), 'mrca_82': Tree node 'OMBQ01000121|117|22' (0x7f89ac340d6), 'mrca_83': Tree node 'HvCF_D5_ms_2|117|76' (0x7f89ac340df), 'mrca_86': Tree node '0.774' (0x7f89ac340f1), 'mrca_119': Tree node 'ERR589547_NODE_77_length_93544_cov_5.56366|125|90' (0x7f89ac2e307), 'mrca_120': Tree node '0.759' (0x7f89acf4597), 'mrca_121': Tree node '' (0x7f89ac3d3e2), 'mrca_122': Tree node '1.000' (0x7f89ac3d3e8), 'mrca_132': Tree node '0.591' (0x7f89ac32ecd), 'mrca_137': Tree node '0.957' (0x7f89ac340a6), 'mrca_138': Tree node '' (0x7f89ac340e2), 'mrca_139': Tree node '0.000' (0x7f89ac340e5), 'mrca_141': Tree node '0.519' (0x7f89ac34f82), 'mrca_142': Tree node '0.772' (0x7f89ac34fb8), 'mrca_143': Tree node '0.755' (0x7f89ac34fcd), 'mrca_144': Tree node '0.761' (0x7f89ac34fe5), 'mrca_145': Tree node '0.916' (0x7f89ac34ff1), 'mrca_149': Tree node '0.644' (0x7f89ac3d3bb), 'mrca_150': Tree node '0.971' (0x7f89ac362e5), 'mrca_151': Tree node '0.983' (0x7f89ac32ef7), 'mrca_

In [22]:
for mrca_id, mrca in monophyletic_clades.items():
    if mrca.up.is_root():
        print("lol")

lol
lol


In [99]:
t.show()

In [32]:
        
# get the most distant node to the mrca
rooted_trees = dict()
for mrca_id, mrca in monophyletic_clades.items():
    bacteria_leaves_dists = {bacteria_leaf:mrca.get_distance(bacteria_leaf) for bacteria_leaf in t.search_nodes(superkingdom="Bacteria")}
    # sort by distance
    bacteria_leaves_dists = {node: distance for node, distance in sorted(bacteria_leaves_dists.items(), key=lambda item: item[1], reverse=True)}
    farthest_bact_leaf = list(bacteria_leaves_dists.keys())[0]
    # root using the farthest Bacteria
    t.set_outgroup(farthest_bact_leaf)
    #print(bacteria_leaves_dists)
    print(mrca_id, farthest_bact_leaf.name)
    #t.show()
    rooted_trees[farthest_bact_leaf.name] = t


mrca_78 WP_188621521.1
mrca_82 WP_188621521.1
mrca_83 WP_188621521.1
mrca_86 WP_188621521.1
mrca_119 WP_188621521.1
mrca_120 WP_188621521.1
mrca_121 WP_188621521.1
mrca_122 WP_188621521.1
mrca_132 WP_188621521.1
mrca_137 WP_188621521.1
mrca_138 WP_188621521.1
mrca_139 WP_188621521.1
mrca_141 WP_188621521.1
mrca_142 WP_188621521.1
mrca_143 WP_188621521.1
mrca_144 WP_188621521.1
mrca_145 WP_188621521.1
mrca_149 WP_188621521.1
mrca_150 WP_188621521.1
mrca_151 WP_188621521.1
mrca_152 WP_188621521.1


In [56]:
rooted_trees

{'MBS5087642.1': Tree node '' (0x7f83f69a876)}

In [73]:
t.show()

In [33]:
taxa_groups = [["crassvirales","NA", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA", "Viruses"],
               ["crassvirales", "new", "NA", "bact_crass_polytomy"],
               ["crassvirales", "new", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new", "NA"],
               ["crassvirales", "new", "Viruses"],
               ["crassvirales", "new", "bact_crass_polytomy"],
               ["crassvirales", "NA", "Viruses"],
               ["crassvirales", "NA", "bact_crass_polytomy"],
               ["crassvirales", "Viruses", "bact_crass_polytomy"],
               ["crassvirales", "new"],
               ["crassvirales", "NA"],
               ["crassvirales", "Viruses"],
               ["crassvirales", "bact_crass_polytomy"],
               ["crassvirales"]]
               
monophyletic_clades_leaves = dict()
monophyletic_clades = dict()
mrcas_ids_to_clean = list()
cont = 0
for taxa_group in reversed(taxa_groups):
    mrcas = t.get_monophyletic(target_attr="superkingdom", values=taxa_group)
    if mrcas:
        for mrca in mrcas:
            cont += 1
            mrca_id = f"mrca_{cont}"
            # get names of the leaves
            mrca_leaves_names = {leaf.name for leaf in mrca.iter_leaves()}
            if monophyletic_clades_leaves:
                for target_mrca, target_leaves_names in monophyletic_clades_leaves.items():
                    tmp = target_leaves_names.union(mrca_leaves_names)
                    if len(tmp) == len(mrca_leaves_names):
                        mrcas_ids_to_clean.append(target_mrca)
                    
            monophyletic_clades_leaves[mrca_id] = mrca_leaves_names
            monophyletic_clades[mrca_id] = mrca
                        
mrcas_ids_to_clean   = list(set(mrcas_ids_to_clean))
for mrca_id in mrcas_ids_to_clean:
    del monophyletic_clades[mrca_id]
    del monophyletic_clades_leaves[mrca_id]


for mrca_id, mrca in monophyletic_clades.items():
    face = RectFace(5,5, "blue","blue", label=mrca_id)
    mrca.add_face(face, 0, "float")
print(monophyletic_clades)

{'mrca_73': Tree node '' (0x7f89ac34097), 'mrca_77': Tree node 'OMBQ01000121|117|22' (0x7f89ac340d6), 'mrca_78': Tree node 'HvCF_D5_ms_2|117|76' (0x7f89ac340df), 'mrca_81': Tree node '0.774' (0x7f89ac340f1), 'mrca_125': Tree node '0.591' (0x7f89ac32ecd), 'mrca_134': Tree node '0.957' (0x7f89ac340a6), 'mrca_135': Tree node '' (0x7f89ac340e2), 'mrca_136': Tree node '0.000' (0x7f89ac340e5), 'mrca_143': Tree node '0.971' (0x7f89ac362e5), 'mrca_144': Tree node '0.644' (0x7f89ac3d3bb), 'mrca_145': Tree node '0.983' (0x7f89ac32ef7), 'mrca_146': Tree node '0.550' (0x7f89ac3d3df)}


In [34]:
t.show()

In [24]:
mrca.get_sisters()

[Tree node 'MBQ0113515.1' (0x7f89acf445b)]